In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import *
import math

In [8]:
data_path = 'weather.csv' 
data = pd.read_csv(data_path)
if 'Unnamed: 0' in data.columns:
    data.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
#Pembersihan Data
def deteksi_outlier_iqr_manual(data):
    outlier_indices = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        Q1 = data[kolom].quantile(0.25)
        Q3 = data[kolom].quantile(0.75)
        IQR = Q3 - Q1
        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR
        outlier_mask = (data[kolom] < batas_bawah) | (data[kolom] > batas_atas)
        outlier_indices[kolom] = data[kolom][outlier_mask].index.tolist()
    return outlier_indices

def imputasi_outlier(data, outlier_indices):
    for kolom, indices in outlier_indices.items():
        median = data[kolom].median()
        data.loc[indices, kolom] = median
    return data

In [5]:
def cari_persentil(data_terurut, persentil):
    k = (len(data_terurut) - 1) * (persentil / 100)
    f = int(k)
    c = f + 1 if k - f > 0 else f
    if f < len(data_terurut) - 1:
        return data_terurut[f] + (k - f) * (data_terurut[c] - data_terurut[f])
    else:
        return data_terurut[f]

def statistik_manual(data):
    hasil = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        nilai = data[kolom].dropna()
        data_terurut = sorted(nilai)
        n = len(data_terurut)

        mean = sum(data_terurut) / n
        median = cari_persentil(data_terurut, 50)
        variansi = sum((x - mean) ** 2 for x in data_terurut) / (n - 1)
        standar_deviasi = variansi ** 0.5

        hasil[kolom] = {
            'nilai mean': mean,
            'nilai median': median,
            'nilai standar deviasi': standar_deviasi,
            'nilai variansi': variansi,
        }
    return pd.DataFrame(hasil)

Kasus 1

Perusahaan menerima beberapa keluhan bahwa prediksi terkait nilai humidity di suatu daerah seringkali tidak tepat. Hal tersebut berakibat pada kurangnya persiapan masyarakat dalam melakukan penyesuaian kondisi termal tertentu. Dapatkah Anda mengecek apakah rata-rata nilai Humidity lebih dari 75?

In [12]:
# Deteksi dan imputasi outlier
outlier_indices_manual = deteksi_outlier_iqr_manual(data)
data = imputasi_outlier(data.copy(), outlier_indices_manual)

# LANGKAH 1
# Hipotesis nol (H0): Rata-rata nilai Humidity (μ) = 75
mu_0 = 75

# LANGKAH 2
# Hipotesis alternatif (H1): Rata-rata nilai Humidity (μ) > 75

# LANGKAH 3
# Tingkat signifikan (α) = 0.05 
alpha = 0.05

# LANGKAH 4
# Kalkulasi nilai T kritis untuk alpha = 0.05 (Asumsi distribusi T)
t_critical = 1.645  # Ini adalah nilai dari tabel distribusi T


# LANGKAH 5
humidity_data = data['Humidity']
sample_mean = sum(humidity_data) / len(humidity_data)

sum_of_squares = sum([(x - sample_mean) ** 2 for x in humidity_data])
sample_std_dev = math.sqrt(sum_of_squares / (len(humidity_data) - 1))

t_statistic = (sample_mean - mu_0) / (sample_std_dev / math.sqrt(len(humidity_data)))

def normal_cdf(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

p_value = 1 - normal_cdf(t_statistic)

print(f"T-Statistic: {t_statistic}")
print(f"P-Value: {p_value}")

#LANGKAH 6
if p_value < alpha:
    print("TOLAK H0: Rata-rata nilai Humidity lebih dari 75.")
else:
    print("Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Humidity lebih dari 75.")

T-Statistic: -23.075432908744677
P-Value: 1.0
Gagal Menolak H0: Tidak ada cukup bukti untuk menyatakan rata-rata nilai Humidity lebih dari 75.
